In [13]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

import sys
sys.path.append('/home/cat/code/gerbil/utils/')

#
import os
import numpy as np

#
from visualize import Visualize
from track import Track


from tqdm import trange

#
import sleap


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [79]:

fname = "/media/cat/256GB/dan/testing_track_cleanup_code/cohort2_night_1000/videos/cohort2_night.1000.slp"
labels = sleap.load_file(fname)

labels.videos  # Use in ipython to print list of videos in the labels object
n_videos = len(labels.videos)

#
huddle_location = np.array([150,150])
min_dist = 100

#
print ("huddle location: ", huddle_location)

#
n_all=0
for video_idx in trange(n_videos):
    #video_idx =   # Change this select the video of interest from the labels.video list
    video = labels.videos[video_idx]
    labeled_frames_from_video = labels.get(video)

    # Loop through all labeled frames (from a specific video)
    n_frames = 0
    for ctr1, lf in enumerate(labeled_frames_from_video):

        # Loop through all user instances in the current labeled frame
        for ctr2, inst in enumerate(lf.user_instances):

            #
            remove_inst = False
            points_array = inst.points_array  # Returns a (num_nodes, 2) np.recarray

            # remove point wise features
            if False:
                diff = points_array - huddle_location
                dists = np.linalg.norm(diff,axis=1)
            
            # this sometimes leaves chimera track in
            if False:
                median = np.nanmedian(points_array,axis=0)
                #median = np.nanmean(points_array,axis=0)
                #print ("median: ", median)
                dist = np.linalg.norm(median-huddle_location)
            
            # this checks for any features at all in the exclusion radius
            if True:
                idx = np.where(np.isnan(points_array.sum(1))==False)[0]
                #print (idx, points_array)
                points_array = points_array[idx]
                #print ("points array fixed: ", points_array)
                
                dist = np.linalg.norm(points_array-huddle_location, axis=1)
                
                dist = np.min(dist)
            
            # delete tracks 
            if dist<min_dist:
                lf.instances.remove(inst)
            #print ('')

# Save the modified slp
new_filename = fname[:-4]+"_deleted.slp"  # Use a different path from the current slp to be safe
labels.save_file(labels, new_filename)
            

huddel location:  [150 150]


100%|██████████| 12/12 [00:01<00:00, 10.69it/s]


In [81]:
####################################################
############ VISUALIZE DELETED TRACKS ##############
####################################################

fname = "/media/cat/256GB/dan/testing_track_cleanup_code/cohort2_night_1000/videos/cohort2_night.1000_deleted.slp"
labels = sleap.load_file(fname)

labels.videos  # Use in ipython to print list of videos in the labels object
n_videos = len(labels.videos)

#
locs = []
for video_idx in trange(n_videos):
    #video_idx =   # Change this select the video of interest from the labels.video list
    video = labels.videos[video_idx]
    labeled_frames_from_video = labels.get(video)

    # Loop through all labeled frames (from a specific video)
    n_frames = 0
    for ctr1, lf in enumerate(labeled_frames_from_video):

        # Loop through all user instances in the current labeled frame
        for ctr2, inst in enumerate(lf.user_instances):

            #
            remove_inst = False
            points_array = inst.points_array  # Returns a (num_nodes, 2) np.recarray
            
            #
            #print (points_array[:,0])
            #print (points_array[:,1])
            idx1 = np.where(points_array[:,0]<200)[0]
            idx2 = np.where(points_array[:,1]<200)[0]
            
            inter = list(set(idx1) & set(idx2))
            locs.append(points_array)
            
            n_all+=1
            n_frames+=1
    print (n_frames)
    
print (n_all)
locs = np.vstack(locs)
print ("locs: ", locs.shape)

#
plt.figure()
plt.scatter(locs[:,0],
            locs[:,1])
plt.xlim(0,900)
plt.ylim(0,700)
plt.show()


 33%|███▎      | 4/12 [00:00<00:00, 25.54it/s]

127
209
171
348
285


 58%|█████▊    | 7/12 [00:00<00:00, 17.38it/s]

456
504
303


 92%|█████████▏| 11/12 [00:00<00:00, 15.53it/s]

586
352
374


100%|██████████| 12/12 [00:00<00:00, 15.90it/s]


500
8430
locs:  (25290, 2)
